Test the `src` package.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os, os.path
import sys
import json
import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colormaps
import torch
import trimesh

sys.path.insert(0, "../")
import src
from src import workspace as ws

# Initialization

In [ ]:
from src.utils import set_seed

seed = 0
expdir = "../experiments/src_test/"
set_seed(seed)
ws.build_experiment_dir(expdir)
specs = ws.load_specs(expdir)

print(f"Running experiment in {expdir}")
print(f"Seeds initialized to {seed}.")

# Data

In [ ]:
from src.data import SdfDataset
from torch.utils.data import DataLoader

batch_size = 8
n_samples = 8192

with open(specs["TrainSplit"]) as f:
    instances = json.load(f)

dataset = SdfDataset(specs["DataSource"], instances, n_samples, 
                     specs["SamplesDir"], specs["SamplesFile"])
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
len_dataset = len(dataset)

print(f"{len_dataset} shapes in training dataset.")

In [ ]:
# Visualize
from matplotlib import cm

idx = 0
filename = dataset.filenames[idx]
idx, points, sdf = dataset[0]
print(f"{len(points)} points for {filename} shape.")

N = 1000
cmap = colormaps['bwr']
c = np.clip(sdf[:N,0], -0.1, 0.1)
vmax = np.abs(c).max()
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(points[:N,0], points[:N,1], points[:N,2], c=c, cmap=cmap, vmin=-vmax, vmax=vmax)
fig.colorbar(p)
ax.set_title("Recon. samples")
fig.tight_layout()

# Model and latents

In [ ]:
from src.model import get_model, get_latents, features

latent_dim = 128
model = get_model(
    "LatentModulatedDeepSDF",
    latent_dim=latent_dim,
    hidden_dim=256,
    n_layers=6,
    dropout=0.,
    activation="relu",
    features = None
).cuda()

latents = get_latents(len(dataset), latent_dim, None)

print(f"Model has {sum([x.nelement() for x in model.parameters()]):,} parameters.")
print(f"{latents.num_embeddings} latent vectors of size {latents.embedding_dim}.")

# Initialize history
history = {'epoch': 0}

# Training

In [ ]:
from src.loss import get_loss_recon
from src.optimizer import get_optimizer, get_scheduler
from src.utils import clamp_sdf

n_epochs = 20
clampD = 0.1

# Loss and optimizer
loss_recon = get_loss_recon("L1-Hard", reduction='none')
latent_reg = 1e-4

optimizer = get_optimizer([model, latents], type="adam", lrs=[0.0005, 0.001])
scheduler = get_scheduler(optimizer, Type="Constant")

# Training
for key in ['loss', 'loss_reg', 'lr', 'lr_lat', 'lat_norm']:
    if key not in history:
        history[key] = []
model.train()
for epoch in range(history['epoch']+1, n_epochs+1):
    time_epoch = time.time()
    running_losses = {'loss': 0., 'loss_reg': 0.}
    optimizer.zero_grad()

    for i, (indices, xyz, sdf_gt) in enumerate(dataloader):
        xyz = xyz.cuda()  # BxNx3
        sdf_gt = sdf_gt.cuda()  # BxNx1
        indices = indices.cuda().unsqueeze(-1).repeat(1, xyz.shape[1])  # BxN
        batch_latents = latents(indices)  # BxNxL

        inputs = torch.cat([batch_latents, xyz], dim=-1)  # BxNx(L+3)
        sdf_pred = model(inputs)
        if clampD is not None and clampD > 0.:
            sdf_pred = clamp_sdf(sdf_pred, clampD, ref=sdf_gt)
            sdf_gt = clamp_sdf(sdf_gt, clampD)

        loss = loss_recon(sdf_pred, sdf_gt).mean()
        running_losses['loss'] += loss.item() * batch_size
        # Latent regularization
        if latent_reg is not None and latent_reg > 0.:
            loss_reg = min(1, epoch / 100) * batch_latents[:,0,:].square().sum()
            loss = loss + latent_reg * loss_reg
            running_losses['loss_reg'] += loss_reg.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    history['epoch'] += 1
    history['loss'].append(running_losses['loss'] / len_dataset)
    history['loss_reg'].append(running_losses['loss_reg'] / len_dataset)
    history["lr"].append(optimizer.state_dict()["param_groups"][0]["lr"])
    history["lr_lat"].append(optimizer.state_dict()["param_groups"][1]["lr"])
    lat_norms = torch.norm(latents.weight.data.detach(), dim=1).cpu()
    history["lat_norm"].append(lat_norms.mean())

    # Apply lr-schedule
    if scheduler is not None:
        scheduler.step()
    
    print(f"Epoch {epoch}/{n_epochs}: loss={loss.item():.6f} - loss_reg={loss_reg.item():.6f}" + \
          f" ({time.time() - time_epoch:.0f}s/epoch)")
torch.cuda.empty_cache()

In [ ]:
# Visualize training history
fig, axs = plt.subplots(1, 4, figsize=(12,3))
axs[0].plot(history['loss'])
axs[0].set_title("Loss")
axs[1].plot(history['loss_reg'])
axs[1].set_title("Reg. loss")
axs[2].plot(history['lr'])
axs[2].plot(history['lr_lat'])
axs[2].legend(['lr', 'lr_lat'])
axs[2].set_title("LRs")
axs[3].plot(history['lat_norm'])
axs[3].set_title("Lat. norm")
for ax in axs.flatten():
    ax.set_xlabel("Epoch")
fig.tight_layout();

# Results

In [ ]:
from sklearn.decomposition import PCA

from src.utils import sample_latents as _sample_latents

_pca = PCA(whiten=True).fit(latents.weight.detach().cpu().numpy())
def sample_latents(n=1, expvar=None):
    """PCA sampling of latent(s) from training distribution."""
    return _sample_latents(latents, n_samples=n, expvar=expvar, pca=_pca)

## Visualization

In [ ]:
# SDF
from src.utils import make_grid2d
from src.mesh import compute_sdf

idx = 0
clamp = True

clamp &= clampD is not None and clampD > 0.
latent = latents(torch.tensor([idx]).cuda()) if idx is not None else sample_latents()
print(f"Clamping at {clampD}." if clamp else "No clamping.")

model.eval()
cmap = colormaps['bwr']
fig, axs = plt.subplots(1, 3, figsize=(14, 3.5))
for i, (ax, ax_name) in enumerate(zip(axs.flatten(), ['x', 'y', 'z'])):
    xyz = make_grid2d([[-1, -1], [1, 1]], 512, i, 0.)
    with torch.no_grad():
        sdf = compute_sdf(model, latent, xyz.cuda()).squeeze().detach().cpu()

    vmax = min(vmax, clampD) if clamp else sdf.abs().max()
    ax.set_title(f"SDF at {ax_name}=0.")
    im = ax.imshow(sdf.T.flip(0), cmap=cmap, vmin=-vmax, vmax=vmax, extent=[-1,1,-1,1])
    plt.colorbar(im, ax=ax)
fig.tight_layout()
fig.show()

In [ ]:
# Mesh
from src.mesh import create_mesh

idx = 0
latent = latents(torch.tensor([idx]).cuda()) if idx is not None else sample_latents()
mesh = create_mesh(model, latent, 256, 32**3, verbose=True)
mesh.show()

In [ ]:
# Rendering
from src import visualization as viz

image = viz.render_mesh(mesh)
plt.imshow(image)

# Test

In [ ]:
# Reconstruction
from src.reconstruct import reconstruct

idx = 0

with open(specs["TestSplit"]) as f:
    instances_t = json.load(f)
instance = instances_t[idx]
print(f"Reconstructing test shape {idx} ({instance})")

filename = os.path.join(specs["DataSource"], specs["SamplesDir"], instance, specs["SamplesFile"])
npz = np.load(filename)

err, latent = reconstruct(model, npz, 400, 8000, 5e-3, loss_recon, latent_reg, clampD, None, latent_dim, verbose=True)
print(f"Final error: {err:.6f}.")
print(f"Latent: norm={latent.norm():.4f} - std={latent.std():.4f}")
test_mesh = create_mesh(model, latent, 256, 32**3, grid_filler=True, verbose=True)

filename = os.path.join(specs["DataSource"], "meshes", instance+".obj")
gt_mesh = trimesh.load(filename)

images = viz.render_meshes([gt_mesh, test_mesh])
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(images[0]); axs[0].set_title("GT")
axs[1].imshow(images[1]); axs[1].set_title("Reconstruction")
fig.show()

In [ ]:
# Metrics
from src.metric import chamfer_distance

chamfer_samples = 30_000

gt_samples = gt_mesh.sample(chamfer_samples)
recon_samples = test_mesh.sample(chamfer_samples)
chamfer_val = chamfer_distance(gt_samples, recon_samples)
print(f"Chamfer-distance = {chamfer_val:.6f}")

In [ ]:
raise RuntimeError("Stop here.")

# Test SDF query speed
Test speed of queriyng SDF (+grad) with a formula, IGL, and a network (DeepSDF like).

In [ ]:
import time

import igl

In [ ]:
# Sphere: formula vs. IGL
def sphere_SDF_grad(xyz):
    norms = np.linalg.norm(xyz, axis=-1, keepdims=True)
    return norms - 0.5, xyz / norms


sphere_mesh = trimesh.creation.uv_sphere(0.5, [32, 32])
print(len(sphere_mesh.faces))
def igl_sphere(xyz):
    sdf, _, _, grads = igl.signed_distance(xyz, sphere_mesh.vertices, sphere_mesh.faces, return_normals=True)
    return sdf, grads

# 800 time-steps, 2048 particles
xyz = np.random.rand(800, 2048, 3) * 2 - 1

formula_times = []
igl_times = []
for t in range(len(xyz)):
    start = time.perf_counter()
    sdf, grads = sphere_SDF_grad(xyz[t])
    formula_times.append(time.perf_counter() - start)
    
    start = time.perf_counter()
    sdf, grads = igl_sphere(xyz[t])
    igl_times.append(time.perf_counter() - start)
sphere_mesh.show()

In [ ]:
print(np.sum(formula_times), np.mean(formula_times), np.std(formula_times))
print(np.sum(igl_times), np.mean(igl_times), np.std(igl_times))
plt.hist(formula_times, bins=20);
plt.hist(igl_times, bins=20);

In [ ]:
# IGL on chairs

chair = trimesh.load(
    "/cvlabsrc1/cvlab/datasets_talabot/ShapeNet/ShapeNetV2_raw/Chair/20ae4b27e86521a32efc7fb40a53aaac/model.obj"
)
chair.show()

print(len(chair.faces))
def igl_chair(xyz):
    sdf, _, _, grads = igl.signed_distance(xyz, chair.vertices, chair.faces, return_normals=True)
    return sdf, grads

# 800 time-steps, 2048 particles
xyz = np.random.rand(800, 2048, 3) * 2 - 1

igl_times2 = []
for t in range(len(xyz)):
    start = time.perf_counter()
    sdf, grads = igl_chair(xyz[t])
    igl_times2.append(time.perf_counter() - start)
print(np.sum(igl_times2), np.mean(igl_times2), np.std(igl_times))
plt.hist(igl_times2);

In [ ]:
# Network (untrained)
mynet = torch.nn.Sequential(
    torch.nn.Linear(3, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 512), torch.nn.ReLU(),
    torch.nn.Linear(512, 1)
).cuda()

xyz = (torch.rand((800, 2048, 3)) * 2 - 1).cuda()

In [ ]:
dnn_times = []
for t in range(len(xyz)):
    start = time.perf_counter()
    xyz_ = xyz[t].requires_grad_()
    sdf = mynet(xyz_)
    grads, = torch.autograd.grad(sdf.sum(), xyz_)
    _ = f"{sdf[0]}"
    dnn_times.append(time.perf_counter() - start)
print(np.sum(dnn_times), np.mean(dnn_times), np.std(dnn_times))
plt.hist(dnn_times, bins=20);

In [ ]:
with torch.autograd.profiler.profile() as prof:
    for t in range(len(xyz)):
        xyz_ = xyz[t].requires_grad_()
        sdf = mynet(xyz_)
        grads, = torch.autograd.grad(sdf.sum(), xyz_)

In [ ]:
# NOTE: some columns were removed for brevity
print(prof.total_average())

In [ ]:
0.003*800

## Copy dataset
Copy a minimal version of the dataset for student projects.

In [ ]:
import shutil

In [ ]:
datasource = "/cvlabsrc1/cvlab/datasets_talabot/shapenet_disn/1_normalized/chairs/"

with open(os.path.join(datasource, "splits/chairs_train1210_benoit.json")) as f:
    instances = json.load(f)
    
with open(os.path.join(datasource, "splits/chairs_test113_benoit.json")) as f:
    instances_t = json.load(f)

full_len = len(instances) + len(instances_t)
print(f"{full_len} chairs to copy.")

os.makedirs(os.path.join(datasource, f"chairs_{full_len}"))
os.makedirs(os.path.join(datasource, f"chairs_{full_len}/meshes"))
os.makedirs(os.path.join(datasource, f"chairs_{full_len}/samples"))
os.makedirs(os.path.join(datasource, f"chairs_{full_len}/splits"))

for instance in instances + instances_t:
    shutil.copyfile(os.path.join(datasource, f"meshes/{instance}.obj"),
                    os.path.join(datasource, f"chairs_{full_len}/meshes/{instance}.obj"))
    
    os.makedirs(os.path.join(datasource, f"chairs_{full_len}/samples/{instance}"))
    shutil.copyfile(os.path.join(datasource, f"samples/{instance}/deepsdf.npz"),
                    os.path.join(datasource, f"chairs_{full_len}/samples/{instance}/deepsdf.npz"))

with open(os.path.join(datasource, f"chairs_{full_len}/splits/chairs_train{len(instances)}.json"), 'w') as f:
    json.dump(instances, f, indent=2)
    
with open(os.path.join(datasource, f"chairs_{full_len}/splits/chairs_test{len(instances_t)}.json"), 'w') as f:
    json.dump(instances_t, f, indent=2)